<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Scenario" data-toc-modified-id="Scenario--1.1">Scenario</a></span></li></ul></li><li><span><a href="#Taxi-Data" data-toc-modified-id="Taxi-Data-0">Taxi Data</a></span></li><li><span><a href="#Data-Cleaning" data-toc-modified-id="Data-Cleaning-1">Data Cleaning</a></span><ul class="toc-item"><li><span><a href="#Columns" data-toc-modified-id="Columns-1.1">Columns</a></span></li><li><span><a href="#Missing-Values" data-toc-modified-id="Missing-Values-1.2">Missing Values</a></span></li><li><span><a href="#Profile-Report" data-toc-modified-id="Profile-Report-1.3">Profile Report</a></span></li><li><span><a href="#Data-Discovery" data-toc-modified-id="Data-Discovery-1.4">Data Discovery</a></span><ul class="toc-item"><li><span><a href="#SNS-Pairplot" data-toc-modified-id="SNS-Pairplot-1.4.1">SNS Pairplot</a></span></li></ul></li></ul></li></ul></div>

## Scenario

Stakeholders: CEO, business analyst

Goal: Prepare data for business analyst by wrangling and cleaning raw data

# Taxi Data

Importing libraries

In [1]:
import pandas as pd
import numpy as np
import math
import time
from datetime import datetime, timedelta

from ydata_profiling import ProfileReport
import missingno as missno 
import calendar

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets

Data Profiling

Importing Raw Data as 'df':

In [2]:
df = pd.read_csv(r"train.csv", header=0)

Summary of dataframe:

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1710670 entries, 0 to 1710669
Data columns (total 9 columns):
 #   Column        Dtype  
---  ------        -----  
 0   TRIP_ID       int64  
 1   CALL_TYPE     object 
 2   ORIGIN_CALL   float64
 3   ORIGIN_STAND  float64
 4   TAXI_ID       int64  
 5   TIMESTAMP     int64  
 6   DAY_TYPE      object 
 7   MISSING_DATA  bool   
 8   POLYLINE      object 
dtypes: bool(1), float64(2), int64(3), object(3)
memory usage: 106.0+ MB


In [4]:
df.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE
0,1372636858620000589,C,NaN,NaN,20000589,1372636858,A,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[..."
1,1372637303620000596,B,NaN,7.0,20000596,1372637303,A,False,"[[-8.639847,41.159826],[-8.640351,41.159871],[..."
2,1372636951620000320,C,NaN,NaN,20000320,1372636951,A,False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-..."
3,1372636854620000520,C,NaN,NaN,20000520,1372636854,A,False,"[[-8.574678,41.151951],[-8.574705,41.151942],[..."
4,1372637091620000337,C,NaN,NaN,20000337,1372637091,A,False,"[[-8.645994,41.18049],[-8.645949,41.180517],[-..."


Note:
(following information from kaggle)

CALL_TYPE: (char) It identifies the way used to demand this service. It may contain one of three possible values:
- ‘A’ if this trip was dispatched from the central;
- ‘B’ if this trip was demanded directly to a taxi driver on a specific stand;
- ‘C’ otherwise (i.e. a trip demanded on a random street).


DAYTYPE: (char) It identifies the daytype of the trip’s start. It assumes one of three possible values:
- ‘A’ otherwise (i.e. a normal day, workday or weekend);
- ‘B’ if this trip started on a holiday or any other special day (i.e. extending holidays, floating holidays, etc.);
- ‘C’ if the trip started on a day before a type-B day;

POLYLINE: (String): It contains a list of GPS coordinates (i.e. WGS84 format) mapped as a string. The beginning and the end of the string are identified with brackets (i.e. [ and ], respectively). Each pair of coordinates is also identified by the same brackets as [LONGITUDE, LATITUDE]. This list contains one pair of coordinates for each 15 seconds of trip. The last list item corresponds to the trip’s destination while the first one represents its start;

# Data Cleaning

## Columns

Changing column names to all lowercase for ease of use

In [5]:
df.columns = map(str.lower, df.columns)

Adding columns:

Timestamp

Changing timestamp column to datetime format

In [6]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

date

In [7]:
df['date'] = df['timestamp'].dt.date

note: all data occurs on the same day

start_time

start_time : the start time of the journey is derived from the time in 'timestamp'

In [8]:
df['start_time'] = df['timestamp'].dt.time

polyline

In [9]:
def find_indices(search_list, search_item):
    indices = []
    for (index, item) in enumerate(search_list):
        if item == search_item:
            indices.append(index)
    return indices

starting_point

This column was created using the polyline column to find the first pair of coordinates. If a pair of coordinates is not present, then it will be presented as 'NA'.

In [10]:
s = []

for i in range(len(df)):
    x = find_indices(df["polyline"][i], ",")
    if len(x) >= 2:
        y = x[1]
        s.append(df["polyline"][i][1:y])
    else:
        s.append("NA")
        
df['starting_point'] = s

ending_point

This column was created using the polyline column to find the last pair of coordinates. If a pair of coordinates is not present, then it will be presented as 'NA'.

In [11]:
e = []

for i in range(len(df)):
    x = find_indices(df["polyline"][i], ",")
    if len(x) >= 2:
        y = x[-2] + 1
        e.append(df["polyline"][i][y:-1])
    else:
        e.append("NA")
    
df['ending_point'] = e

distance

Distance was found using pythagoras theorem: a² + b² = c²

a = x2 - x1

b = y2 - y1

x2 and x1: the x coordinates of the starting and ending point
y2 and y1: the y coordinates of the starting and ending point

Steps:
- the x and y values of the starting and ending point had to be found
- the absolute difference between the x values were found (repeat for y values)
- find c using previous results
- add to new column 'distance'

In [12]:
ld = []

for i in range(len(df)):
    sp = df['starting_point'][i].split(",")
    ep = df['ending_point'][i].split(",")
    if len(sp) == 2  and len(ep) == 2:
        x1 = sp[0][1:]
        y1 = sp[1][:-1]
        x2 = ep[0][1:]
        y2 = ep[1][:-1]
        if any(i.isdigit() for i in x1) == True and any(i.isdigit() for i in x2) == True:
            a = abs(float(x2) - float(x1))
            b = abs(float(y2) - float(y1))
            c = math.sqrt(a**2 + b**2)
        else:
            c = "Nan"
    else:
        c = "NaN"
    ld.append(c)
    

df["distance"] = ld

duration

note: every addiontional coordinate from polyline column represents 15 seconds. E.g. 3 coordinates = 30 seconds

Steps:
- find number of "," in cell
- multiply value by 15

In [13]:
t = []

for i in range(len(df)):
    x = find_indices(df["polyline"][i], ",")
    c = len(x)
    t.append(c*15)
    
    
df["duration"] = t

end_time

end time is found by taking start time and adding duration

In [14]:
df["end_timestamp"] = df["timestamp"] + pd.to_timedelta(df['duration'], unit='s')

df['end_time'] = df['end_timestamp'].dt.time

speed

speed = distance / time

In [15]:
df["distance"] = pd.to_numeric(df["distance"], errors="coerce")
df["duration"] = pd.to_numeric(df["duration"], errors="coerce")

In [16]:
df["speed"] = df["distance"] / df["duration"]

Changing column order

Changing column order to a more logical order. Putting time related columns next to each other as well as coordinates and distance related columns together.

Also removing column 'timestamp' as the same information is in columns 'date' and 'start_time'.

In [17]:
new_order = ["trip_id","call_type","origin_call","origin_stand","taxi_id","day_type","start_time","end_time","date","duration","speed","polyline","starting_point","ending_point","distance","missing_data"]

df = df.reindex(columns=new_order)

## Missing Values

Profiling:

In [18]:
print("Total missing value in dataframe:")
df.isnull().sum().sum()

Total missing value in dataframe:


2323011

In [19]:
print("Total missing value by column:")
df.isnull().sum()

Total missing value by column:


trip_id                 0
call_type               0
origin_call       1345900
origin_stand       904091
taxi_id                 0
day_type                0
start_time              0
end_time                0
date                    0
duration                0
speed               36510
polyline                0
starting_point          0
ending_point            0
distance            36510
missing_data            0
dtype: int64

In [20]:
print("Total rows with no missing entries in all their columns:")

complete_rows = df.isnull().sum(axis=1) == 0
num_complete_rows = len(df[complete_rows])

print(num_complete_rows)

Total rows with no missing entries in all their columns:
0


In [21]:
print("Missing values in Polyline:")
sum(len(row['polyline']) < 19 for _, row in df.iterrows())

Missing values in Polyline:


5903

note: 19 was used as that is the minimum length for one set of coordinates (i.e [0.000000,0.000000] )

Replacing missing values

polyline

If lenght of polyline is less than 19, then change value to 'None'

In [22]:
for _, row in df.iterrows():
    if len(row['polyline']) < 20:
        row['polyline'] = 'None'
    else:
        continue

missing_data

Changing values from true to false if polyline = 'None':

In [ ]:
for _, row in df.iterrows():
    if row['polyline'] == 'None':
        row['missing_data'] = 'True'
    else:
        row['missing_data'] = 'False'

Missingno

In [ ]:
missno.bar(df, log=True) 

missno.heatmap(df) 

missno.dendrogram(df)

Duplicated Rows

In [ ]:
duplicated_rows = df.duplicated()   

In [ ]:
print("Number of duplicated rows:")
sum(duplicated_rows)

In [ ]:
print(df[duplicated_rows]) 

Remove duplicated rows

In [ ]:
df.drop_duplicates(inplace=False)

Note: raw data had 1710670 rows, now there are 3 fewer.

Column Values

missing_data

Change values to all lowercase

In [ ]:
df['missing_data'] = df['missing_data'].replace({'True': 'true', 'False': 'false'})

call_type

check that all values are either A, B, or C

In [ ]:
print(df['call_type'].unique())

day_type

check that all values are either A, B, or C

In [ ]:
print(df['day_type'].unique())

Column Type

In [ ]:
df.info()

In [ ]:
df.head()

Changing column dtype

Changing column types to match values.
- trip_id = string
- taxi_id = string

both columns are identification values, so string is more appropriate than integer

In [ ]:
df['trip_id'] = df['trip_id'].astype(str)

In [ ]:
df['taxi_id'] = df['taxi_id'].astype(str)

Final dtypes

In [ ]:
df.info()

## Profile Report

Note: dataframe is too large to profile report the whole dataframe, so 1000 random rows will be selected. random_state is set to ensure that the same random sample is produced when the code is run

In [ ]:
sample_size = 1000
random_sample = df.sample(n=sample_size, random_state=42) 

profile = ProfileReport(random_sample) 

In [ ]:
profile

In [ ]:
profile.to_file(output_file='taxi_profile.html')

Saving Dataframe:

Saving dataframe for further analysis:

In [ ]:
df.to_csv('taxi.csv', index=False)

## Data Discovery

In [ ]:
df = pd.read_csv(r"taxi.csv", header=0)

Pie Chart

In [ ]:
df['call_type'].value_counts().plot(kind='pie', figsize=(5,5))
plt.legend()      
plt.title("Call Type")
plt.show

Scatter Plot

Duration by Distance

In [ ]:
plt.scatter(df['duration'],df['distance'])
plt.xlabel("Duration (sec)")
plt.ylabel("Distance")
plt.title('Duration vs Distance')
plt.show

Starting Point

Steps to find x and y values from coordinate:
- split starting_point column by "," to find x and y coordinate
- remove first character of x coordinate and last character of y coordinate as they are "[" and "]" respectively

In [ ]:
df[['x_start', 'y_start']] = df['starting_point'].str.split(',', expand=True)

In [ ]:
df['x_start'] = pd.to_numeric(df['x_start'].str[1:], errors='coerce')
df['y_start'] = pd.to_numeric(df['y_start'].str[:-1], errors='coerce')

In [ ]:
plt.scatter(df['x_start'],df['y_start'])
plt.xlabel("X Coordinate")
plt.ylabel("Y Coordinate")
plt.title('Starting Point')
plt.xlim(-14, -4)
plt.ylim(36,52)
plt.show

Ending Point

Same method is used for starting point

In [ ]:
df['ending_point'].str.split(',')

df[['x_end', 'y_end']] = df['ending_point'].str.split(',', expand=True)

In [ ]:
df['x_end'] = pd.to_numeric(df['x_end'].str[1:], errors='coerce')
df['y_end'] = pd.to_numeric(df['y_end'].str[:-1], errors='coerce')

In [ ]:
plt.scatter(df['x_end'],df['y_end'])
plt.xlabel("X Coordinate")
plt.ylabel("Y Coordinate")
plt.title('Ending Point')
plt.xlim(-14, -4)
plt.ylim(36,52)
plt.show

Starting and Ending Point

In [ ]:
plt.scatter(df['x_start'],df['y_start'], c='b', marker='X', alpha=0.5)
plt.scatter(df['x_end'],df['y_end'], c='r', marker='+', alpha=0.5)
plt.xlabel("X Coordinate")
plt.ylabel("Y Coordinate")
plt.legend()
plt.title('Starting and Ending Points')
plt.show

Histogram

In [ ]:
bin_range = range(0, 8000, 250)

In [ ]:
plt.hist(df['duration'],  bins=20, range=(0, 8000))
plt.title('Duration')
plt.xlabel('Duration (secs)')
plt.ylabel('Frequency')
plt.show()

Log10

In [ ]:
duration_skew = df['duration'].skew()
duration_kurt = df['duration'].kurt()

In [ ]:
non_zero = df['duration'] != 0


df['duration_log'] = np.log10(df['duration'][non_zero])

In [ ]:
log10_duration_skew = df['duration_log'].skew()
log10_duration_kurt = df['duration_log'].kurt()

In [ ]:
print("Ideal Skew :  0")
print("Ideal Kurt :  3")
print('Duration Skew : ' + str(duration_skew))
print('Duration Kurt : ' + str(duration_kurt))
print('Duration Log10 Skew : ' + str(log10_duration_skew))
print('Duration Log10 Kurt : ' + str(log10_duration_kurt))

In [ ]:
plt.hist(df['duration_log'], bins=20, range=(1, 5))
plt.title('Duration Log10')
plt.xlabel('Duration (log)')
plt.ylabel('Frequency')
plt.show()

Boxplot

In [ ]:
plt.boxplot(df['duration'], showfliers=False, vert= False)
plt.title("Duration of Journey")
plt.xlabel("Duration (sec)", fontsize=12)
plt.show()

Line Graph

Plotting a journey using the coordinates from polyline column

note: can't plot on map because it's test data (i.e coordinates aren't real)

In [ ]:
c = str(df['polyline'][0])

cx = c.split(',')[::2]
cy = c.split(',')[1::2]

x = []
y = []

for c in range(len(cx)):
    x.append(cx[c][1:])
    y.append(cy[c][:-1])
    
x[0] = x[0][1:]
y[-1] = y[-1][:-1]

In [ ]:
plt.plot(x,y, marker = ('+'))
plt.xticks(rotation=45)
plt.title('Journey of ride (0)')
plt.xlabel('X Coordinates')
plt.ylabel('Y Coordinates')
plt.show()

In [ ]:
c = str(df['polyline'][10000])

cx = c.split(',')[::2]
cy = c.split(',')[1::2]

x = []
y = []

for c in range(len(cx)):
    x.append(cx[c][1:])
    y.append(cy[c][:-1])
    
x[0] = x[0][1:]
y[-1] = y[-1][:-1]

In [ ]:
plt.plot(x,y, marker = ('+'))
plt.xticks(rotation=45)
plt.title('Journey of ride (10000)')
plt.xlabel('X Coordinates')
plt.ylabel('Y Coordinates')
plt.show()

### SNS Pairplot

In [ ]:
sns.pairplot(random_sample)
plt.show()